### LS/DR9 Photometry for DESI/EDR

This notebook briefly shows how to work with the LS/DR9 value-added catalogs for a small set of targets in the DESI Early Data Release (EDR).

John Moustakas  
Siena College  
2022 June XX

In [1]:
import os
import numpy as np
import fitsio
from astropy.table import Table, vstack

#### Specify the location of the DESI/EDR redshift catalogs and the LS/DR9 VAC.

In [2]:
# temporary locations
zcatdir = '/Users/ioannis/work/desi/spectro/redux/fuji/zcatalog'
vacdir = '/Users/ioannis/work/desi/ioannis/lsdr9-photometry/v1.0'
#zcatdir = '/global/cfs/cdirs/desi/public/edr/spectro/redux/fuji/zcatalog'
#vacdir = '/global/cscratch1/sd/ioannis/lsdr9-photometry/v1.0'

#### Select a small number of LRG targets from the SV3 (One Percent Survey).

To select these targets, we apply the appropriate targeting bitmask and we also require `ZWARN==0` and `0.700<Z<0.701` (to keep the number of targets relatively small).

In [3]:
def select_targets():
    """Select a small number of targets."""
    from desitarget.sv3.sv3_targetmask import desi_mask

    zcatfile = os.path.join(zcatdir, 'ztile-sv3-dark-cumulative.fits')
    zcat = Table(fitsio.read(zcatfile, 'ZCATALOG'))
    I = ((zcat['SV3_DESI_TARGET'] & desi_mask.mask('LRG') != 0) * 
         (zcat['ZWARN'] == 0) * (zcat['Z'] > 0.7) * (zcat['Z'] < 0.701))
    zcat = zcat[I]
    _, uindx = np.unique(zcat['TARGETID'], return_index=True)
    zcat = zcat[uindx]
    print('Selected {} unique LRG targets from {}'.format(len(zcat), zcatfile))
    return zcat

In [4]:
zcat = select_targets()
zcat

Selected 197 unique LRG targets from /Users/ioannis/work/desi/spectro/redux/fuji/zcatalog/ztile-sv3-dark-cumulative.fits


TARGETID,LASTNIGHT,SPGRPVAL,Z,ZERR,ZWARN,CHI2,COEFF [10],NPIXELS,SPECTYPE,SUBTYPE,NCOEFF,DELTACHI2,PETAL_LOC,DEVICE_LOC,LOCATION,FIBER,COADD_FIBERSTATUS,TARGET_RA,TARGET_DEC,PMRA,PMDEC,REF_EPOCH,LAMBDA_REF,FA_TARGET,FA_TYPE,OBJTYPE,FIBERASSIGN_X,FIBERASSIGN_Y,PRIORITY,SUBPRIORITY,OBSCONDITIONS,RELEASE,BRICKNAME,BRICKID,BRICK_OBJID,MORPHTYPE,EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,FLUX_IVAR_W1,FLUX_IVAR_W2,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,MASKBITS,SERSIC,SHAPE_R,SHAPE_E1,SHAPE_E2,REF_ID,REF_CAT,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_RP_MEAN_MAG,PARALLAX,PHOTSYS,PRIORITY_INIT,NUMOBS_INIT,SV3_DESI_TARGET,SV3_BGS_TARGET,SV3_MWS_TARGET,SV3_SCND_TARGET,DESI_TARGET,BGS_TARGET,MWS_TARGET,PLATE_RA,PLATE_DEC,TILEID,COADD_NUMEXP,COADD_EXPTIME,COADD_NUMNIGHT,COADD_NUMTILE,MEAN_DELTA_X,RMS_DELTA_X,MEAN_DELTA_Y,RMS_DELTA_Y,MEAN_FIBER_RA,STD_FIBER_RA,MEAN_FIBER_DEC,STD_FIBER_DEC,MEAN_PSF_TO_FIBER_SPECFLUX,MEAN_FIBER_X,MEAN_FIBER_Y,TSNR2_GPBDARK_B,TSNR2_ELG_B,TSNR2_GPBBRIGHT_B,TSNR2_LYA_B,TSNR2_BGS_B,TSNR2_GPBBACKUP_B,TSNR2_QSO_B,TSNR2_LRG_B,TSNR2_GPBDARK_R,TSNR2_ELG_R,TSNR2_GPBBRIGHT_R,TSNR2_LYA_R,TSNR2_BGS_R,TSNR2_GPBBACKUP_R,TSNR2_QSO_R,TSNR2_LRG_R,TSNR2_GPBDARK_Z,TSNR2_ELG_Z,TSNR2_GPBBRIGHT_Z,TSNR2_LYA_Z,TSNR2_BGS_Z,TSNR2_GPBBACKUP_Z,TSNR2_QSO_Z,TSNR2_LRG_Z,TSNR2_GPBDARK,TSNR2_ELG,TSNR2_GPBBRIGHT,TSNR2_LYA,TSNR2_BGS,TSNR2_GPBBACKUP,TSNR2_QSO,TSNR2_LRG,ZCAT_NSPEC,ZCAT_PRIMARY
int64,int32,int32,float64,float64,int64,float64,float64,int64,str6,str20,int64,float64,int16,int32,int64,int32,int32,float64,float64,float32,float32,float32,float32,int64,uint8,str3,float32,float32,int32,float64,int32,int16,str8,int32,int32,str4,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,int64,str2,float32,float32,float32,float32,str1,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,int32,int16,float32,int16,int16,float32,float32,float32,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,bool
39627739967261315,20210506,20210506,0.7009097160352763,6.323833382421855e-05,0,7969.83309815824,175.59013750253197 .. -6.649367995358885,7778,GALAXY,,10,372.64027915894985,1,376,1376,547,0,212.1196069212658,-1.881947568934596,0.0,0.0,2015.5,5400.0,2162697,1,TGT,143.31711,-335.3371,103200,0.5817583700335753,1,9010,2121m020,319696,2691,DEV,0.05773544,1.2918397,4.1348476,15.130805,58.938457,42.135635,415.40863,179.9975,24.548807,1.8652909,0.48777327,0.43902764,1.4052149,5.1421566,0.43902767,1.4052149,5.1421566,0,4.0,1.2096204,0.12796898,-0.21814717,0,,0.0,0.0,0.0,0.0,S,103200,9,2162697,0,0,0,0,0,0,212.1196069212658,-1.881947568934596,514,1,1308.3065,1,1,0.004,0.004,0.004,0.004,212.11962302396537,0.0,-1.8819629927097279,0.0,0.7421875,143.282,-335.287,260.04184,0.107650764,48.60683,61.00478,627.50836,331.05646,2.8074384,1.2580719,13586.124,31.521708,2372.675,0.04040304,2634.4824,13789.933,9.192499,44.645294,2.778885e-05,96.61443,5.1076454e-06,0.0,4614.4614,3.353519e-05,20.666698,46.948204,13846.166,128.24379,2421.282,61.04518,7876.452,14120.989,32.666634,92.85157,2,True
39627739967263965,20210610,20210610,0.7007750853865882,9.475305644425126e-05,0,7833.674790084362,125.33596180710313 .. 3.012575849983678,7806,GALAXY,,10,249.92008470743895,1,421,1421,549,0,212.24986395305368,-2.0323299906406485,0.0,0.0,2015.5,5400.0,2162697,1,TGT,161.1864,-341.50107,2,0.8546272238554111,1,9010,2121m020,319696,5341,DEV,0.05601391,0.29178575,2.1601152,9.906716,34.566994,21.584663,690.2582,299.89017,40.563972,2.40093,0.61147714,0.1325734,0.9814524,4.501135,0.13257352,0.98145276,4.5011363,0

#### Define a couple convenience functions.

In [5]:
def read_targetphot(zcat, survey=None):
    """Read the targeting photometry for an input catalog of objects.   

    """
    targetfile = os.path.join(vacdir, 'targetphot-{}-edr.fits'.format(survey))
    alltargetids = fitsio.read(targetfile, 'TARGETPHOT', columns='TARGETID')
    I = np.where(np.isin(alltargetids, zcat['TARGETID']))[0]
    
    targ = Table(fitsio.read(targetfile, rows=I))
    print('Read targeting photometry for {} objects from {}'.format(len(targ), targetfile))
    
    return targ   

In [6]:
def read_tractorphot(zcat, verbose=False):
    from glob import glob
    from desimodel.footprint import radec2pix

    tractorphotfiles = glob(os.path.join(vacdir, 'tractorphot', 'tractorphot-nside4-hp???-edr.fits'))
    
    hdr = fitsio.read_header(tractorphotfiles[0], 'TRACTORPHOT')
    tractorphot_nside = hdr['FILENSID']

    pixels = radec2pix(tractorphot_nside, zcat['TARGET_RA'], zcat['TARGET_DEC'])
    phot = []
    for pixel in set(pixels):
        J = pixel == pixels
        photfile = os.path.join(vacdir, 'tractorphot', 'tractorphot-nside4-hp{:03d}-edr.fits'.format(pixel))
        targetids = fitsio.read(photfile, columns='TARGETID')
        K = np.where(np.isin(targetids, zcat['TARGETID'][J]))[0]
        
        if verbose:
            print('Reading photometry for {} objects from {}'.format(len(K), photfile))
            
        _phot = fitsio.read(photfile, rows=K)
        phot.append(Table(_phot))
    phot = vstack(phot)

    # Is there a better way to sort here??
    srt = np.hstack([np.where(tid == phot['TARGETID'])[0] for tid in zcat['TARGETID']]) 
    phot = phot[srt]
    
    return phot

In [7]:
targ = read_targetphot(zcat, survey='sv3')
targ

Read targeting photometry for 197 objects from /Users/ioannis/work/desi/ioannis/lsdr9-photometry/v1.0/targetphot-sv3-edr.fits


RELEASE,BRICKID,BRICKNAME,BRICK_OBJID,MORPHTYPE,RA,RA_IVAR,DEC,DEC_IVAR,DCHISQ [5],EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,FRACFLUX_G,FRACFLUX_R,FRACFLUX_Z,FRACMASKED_G,FRACMASKED_R,FRACMASKED_Z,FRACIN_G,FRACIN_R,FRACIN_Z,NOBS_G,NOBS_R,NOBS_Z,PSFDEPTH_G,PSFDEPTH_R,PSFDEPTH_Z,GALDEPTH_G,GALDEPTH_R,GALDEPTH_Z,FLUX_W1,FLUX_W2,FLUX_W3,FLUX_W4,FLUX_IVAR_W1,FLUX_IVAR_W2,FLUX_IVAR_W3,FLUX_IVAR_W4,MW_TRANSMISSION_W1,MW_TRANSMISSION_W2,MW_TRANSMISSION_W3,MW_TRANSMISSION_W4,ALLMASK_G,ALLMASK_R,ALLMASK_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,REF_EPOCH,WISEMASK_W1,WISEMASK_W2,MASKBITS,LC_FLUX_W1 [15],LC_FLUX_W2 [15],LC_FLUX_IVAR_W1 [15],LC_FLUX_IVAR_W2 [15],LC_NOBS_W1 [15],LC_NOBS_W2 [15],LC_MJD_W1 [15],LC_MJD_W2 [15],SHAPE_R,SHAPE_E1,SHAPE_E2,SHAPE_R_IVAR,SHAPE_E1_IVAR,SHAPE_E2_IVAR,SERSIC,SERSIC_IVAR,REF_ID,REF_CAT,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_G_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_BP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_RP_MEAN_MAG,GAIA_PHOT_RP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_RP_EXCESS_FACTOR,GAIA_ASTROMETRIC_EXCESS_NOISE,GAIA_DUPLICATED_SOURCE,GAIA_ASTROMETRIC_SIGMA5D_MAX,GAIA_ASTROMETRIC_PARAMS_SOLVED,PARALLAX,PARALLAX_IVAR,PMRA,PMRA_IVAR,PMDEC,PMDEC_IVAR,PHOTSYS,TARGETID,SUBPRIORITY,OBSCONDITIONS,PRIORITY_INIT,NUMOBS_INIT,HPXPIXEL,CMX_TARGET,DESI_TARGET,BGS_TARGET,MWS_TARGET,SV1_DESI_TARGET,SV1_BGS_TARGET,SV1_MWS_TARGET,SV2_DESI_TARGET,SV2_BGS_TARGET,SV2_MWS_TARGET,SV3_DESI_TARGET,SV3_BGS_TARGET,SV3_MWS_TARGET,SCND_TARGET,SV1_SCND_TARGET,SV2_SCND_TARGET,SV3_SCND_TARGET
int16,int32,str8,int32,str4,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,uint8,uint8,int16,float32,float32,float32,float32,int16,int16,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,int64,str2,float32,float32,float32,float32,float32,float32,float32,float32,bool,float32,bool,float32,float32,float32,float32,float32,float32,str1,int64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
9010,319696,2121m020,2691,DEV,212.1196069212658,117733280000.0,-1.881947568934596,96762360000.0,7878.716 .. 9403.314,0.05773544,1.2918397,4.1348476,15.130805,415.40863,179.9975,24.548807,0.8428986,0.89125323,0.93763316,0.020383004,0.008558611,0.004051246,0.04887721,0.04775305,0.053740904,0.9863482,0.9856684,0.985165,3,3,3,1457.567,664.5617,101.641754,955.3136,426.26358,57.57113,58.938457,42.135635,72.84395,-898.98035,1.8652909,0.48777327,0.0013256697,2.0390364e-05,0.9902633,0.9940091,0.9987193,0.9995162,0,0,0,0.43902764,1.4052149,5.1421566,0.43902767,1.4052149,5.1421566,0.0,0,0,0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0 .. 0,0 .. 0,0.0 .. 0.0,0.0 .. 0.0,1.2096204,0.12796898,-0.21814717,1583.4012,1780.8395,2452.538,4.0,0.0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0,False,0.0,0.0,0.0,0.0,0.0,0.0,S,39627739967261315,0.3597789684195404,1,103200,9,25915,0,0,0,0,0,0,0,0,0,0,2162697,0,0,0,0,0,0
9010,319696,2121m020,5341,DEV,212.24986395305368,78958410000.0,-2.0323299906406485,81136670000.0,4990.2866 .. 5453.0654,0.05601391,0.29178575,2.1601152,9.906716,690.2582,299.89017,40.563972,0.84720504,0.894318,0.93943524,0.021313665,0.005943152,0.0024521064,0.009436623,0.009811765,0.01675675,0.97404945,0.9877173,0.99921656,3,3,3,1462.3547,679.6659,105.44706,963.3969,436.3574,59.85231,34.566994,21.584663,44.55249,965.37537,2.40093,0.61147714,0.0013493517,1.9717492e-05,0.99055225,0.99418724,0.9987574,0.9995306,0,0,0,0.1325734,0.9814524,4.501135,0.13257352,0.98145276,4.5011363,0.0,0,0

In [8]:
tractor = read_tractorphot(zcat, verbose=True)
tractor

Reading photometry for 3 objects from /Users/ioannis/work/desi/ioannis/lsdr9-photometry/v1.0/tractorphot/tractorphot-nside4-hp035-edr.fits
Reading photometry for 17 objects from /Users/ioannis/work/desi/ioannis/lsdr9-photometry/v1.0/tractorphot/tractorphot-nside4-hp036-edr.fits
Reading photometry for 2 objects from /Users/ioannis/work/desi/ioannis/lsdr9-photometry/v1.0/tractorphot/tractorphot-nside4-hp037-edr.fits
Reading photometry for 23 objects from /Users/ioannis/work/desi/ioannis/lsdr9-photometry/v1.0/tractorphot/tractorphot-nside4-hp038-edr.fits
Reading photometry for 22 objects from /Users/ioannis/work/desi/ioannis/lsdr9-photometry/v1.0/tractorphot/tractorphot-nside4-hp039-edr.fits
Reading photometry for 6 objects from /Users/ioannis/work/desi/ioannis/lsdr9-photometry/v1.0/tractorphot/tractorphot-nside4-hp040-edr.fits
Reading photometry for 7 objects from /Users/ioannis/work/desi/ioannis/lsdr9-photometry/v1.0/tractorphot/tractorphot-nside4-hp041-edr.fits
Reading photometry for 9

RELEASE,BRICKID,BRICKNAME,OBJID,BRICK_PRIMARY,MASKBITS,FITBITS,TYPE,RA,DEC,RA_IVAR,DEC_IVAR,BX,BY,DCHISQ [5],EBV,MJD_MIN,MJD_MAX,REF_CAT,REF_ID,PMRA,PMDEC,PARALLAX,PMRA_IVAR,PMDEC_IVAR,PARALLAX_IVAR,REF_EPOCH,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_G_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_G_N_OBS,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_BP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_N_OBS,GAIA_PHOT_RP_MEAN_MAG,GAIA_PHOT_RP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_RP_N_OBS,GAIA_PHOT_VARIABLE_FLAG,GAIA_ASTROMETRIC_EXCESS_NOISE,GAIA_ASTROMETRIC_EXCESS_NOISE_SIG,GAIA_ASTROMETRIC_N_OBS_AL,GAIA_ASTROMETRIC_N_GOOD_OBS_AL,GAIA_ASTROMETRIC_WEIGHT_AL,GAIA_DUPLICATED_SOURCE,GAIA_A_G_VAL,GAIA_E_BP_MIN_RP_VAL,GAIA_PHOT_BP_RP_EXCESS_FACTOR,GAIA_ASTROMETRIC_SIGMA5D_MAX,GAIA_ASTROMETRIC_PARAMS_SOLVED,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_W3,FLUX_W4,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,FLUX_IVAR_W1,FLUX_IVAR_W2,FLUX_IVAR_W3,FLUX_IVAR_W4,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,APFLUX_G [8],APFLUX_R [8],APFLUX_Z [8],APFLUX_RESID_G [8],APFLUX_RESID_R [8],APFLUX_RESID_Z [8],APFLUX_BLOBRESID_G [8],APFLUX_BLOBRESID_R [8],APFLUX_BLOBRESID_Z [8],APFLUX_IVAR_G [8],APFLUX_IVAR_R [8],APFLUX_IVAR_Z [8],APFLUX_MASKED_G [8],APFLUX_MASKED_R [8],APFLUX_MASKED_Z [8],APFLUX_W1 [5],APFLUX_W2 [5],APFLUX_W3 [5],APFLUX_W4 [5],APFLUX_RESID_W1 [5],APFLUX_RESID_W2 [5],APFLUX_RESID_W3 [5],APFLUX_RESID_W4 [5],APFLUX_IVAR_W1 [5],APFLUX_IVAR_W2 [5],APFLUX_IVAR_W3 [5],APFLUX_IVAR_W4 [5],MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,MW_TRANSMISSION_W1,MW_TRANSMISSION_W2,MW_TRANSMISSION_W3,MW_TRANSMISSION_W4,NOBS_G,NOBS_R,NOBS_Z,NOBS_W1,NOBS_W2,NOBS_W3,NOBS_W4,RCHISQ_G,RCHISQ_R,RCHISQ_Z,RCHISQ_W1,RCHISQ_W2,RCHISQ_W3,RCHISQ_W4,FRACFLUX_G,FRACFLUX_R,FRACFLUX_Z,FRACFLUX_W1,FRACFLUX_W2,FRACFLUX_W3,FRACFLUX_W4,FRACMASKED_G,FRACMASKED_R,FRACMASKED_Z,FRACIN_G,FRACIN_R,FRACIN_Z,ANYMASK_G,ANYMASK_R,ANYMASK_Z,ALLMASK_G,ALLMASK_R,ALLMASK_Z,WISEMASK_W1,WISEMASK_W2,PSFSIZE_G,PSFSIZE_R,PSFSIZE_Z,PSFDEPTH_G,PSFDEPTH_R,PSFDEPTH_Z,GALDEPTH_G,GALDEPTH_R,GALDEPTH_Z,NEA_G,NEA_R,NEA_Z,BLOB_NEA_G,BLOB_NEA_R,BLOB_NEA_Z,PSFDEPTH_W1,PSFDEPTH_W2,PSFDEPTH_W3,PSFDEPTH_W4,WISE_COADD_ID,WISE_X,WISE_Y,LC_FLUX_W1 [15],LC_FLUX_W2 [15],LC_FLUX_IVAR_W1 [15],LC_FLUX_IVAR_W2 [15],LC_NOBS_W1 [15],LC_NOBS_W2 [15],LC_FRACFLUX_W1 [15],LC_FRACFLUX_W2 [15],LC_RCHISQ_W1 [15],LC_RCHISQ_W2 [15],LC_MJD_W1 [15],LC_MJD_W2 [15],LC_EPOCH_INDEX_W1 [15],LC_EPOCH_INDEX_W2 [15],SERSIC,SERSIC_IVAR,SHAPE_R,SHAPE_R_IVAR,SHAPE_E1,SHAPE_E1_IVAR,SHAPE_E2,SHAPE_E2_IVAR,LS_ID,TARGETID
int16,int32,str8,int32,bool,int16,int16,str3,float64,float64,float32,float32,float32,float32,float32,float32,float64,float64,str2,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,int16,float32,float32,int16,bool,float32,float32,int16,int16,float32,bool,float32,float32,float32,float32,uint8,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,int16,int16,uint8,uint8,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,str8,float32,float32,float32,float32,float32,float32,int16,int16,float32,float32,float32,float32,float64,float64,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,int64,int64
9010,319696,2121m020,2691,True,0,0,DEV,212.1196069212658,-1.881947568934596,117733280000.0,967623